### Importing Libraries

In [3]:
pip install transformers

In [4]:
pip install accelerate==0.21.0

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
pip install datasets evaluate rouge_score

In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [2]:
import torch
import pandas as pd
import os
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer
import numpy as np


### Generating Dataset

In [6]:
!unzip '/content/drive/MyDrive/archive (6).zip'

Streaming output truncated to the last 5000 lines.
  inflating: BBC News Summary/Summaries/sport/363.txt  
  inflating: BBC News Summary/Summaries/sport/364.txt  
  inflating: BBC News Summary/Summaries/sport/365.txt  
  inflating: BBC News Summary/Summaries/sport/366.txt  
  inflating: BBC News Summary/Summaries/sport/367.txt  
  inflating: BBC News Summary/Summaries/sport/368.txt  
  inflating: BBC News Summary/Summaries/sport/369.txt  
  inflating: BBC News Summary/Summaries/sport/370.txt  
  inflating: BBC News Summary/Summaries/sport/371.txt  
  inflating: BBC News Summary/Summaries/sport/372.txt  
  inflating: BBC News Summary/Summaries/sport/373.txt  
  inflating: BBC News Summary/Summaries/sport/374.txt  
  inflating: BBC News Summary/Summaries/sport/375.txt  
  inflating: BBC News Summary/Summaries/sport/376.txt  
  inflating: BBC News Summary/Summaries/sport/377.txt  
  inflating: BBC News Summary/Summaries/sport/378.txt  
  inflating: BBC News Summary/Summaries/sport/379.txt

In [4]:
directory_path = '/content/BBC News Summary/News Articles/sport'

# Initialize lists to store data
file_names = []
file_contents = []

# Loop through files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        file_names.append(file_name)
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'rb') as file:
            content = file.read().decode(errors='ignore')
            file_contents.append(content)

# Create a DataFrame
data = {'File Name': file_names, 'File_Content': file_contents}
df1 = pd.DataFrame(data)


In [5]:
df1['File_Content'][3]

'Dibaba breaks 5,000m world record\n\nEthiopia\'s Tirunesh Dibaba set a new world record in winning the women\'s 5,000m at the Boston Indoor Games.\n\nDibaba won in 14 minutes 32.93 seconds to erase the previous world indoor mark of 14:39.29 set by another Ethiopian, Berhane Adera, in Stuttgart last year. But compatriot Kenenisa Bekele\'s record hopes were dashed when he miscounted his laps in the men\'s 3,000m and staged his sprint finish a lap too soon. Ireland\'s Alistair Cragg won in 7:39.89 as Bekele battled to second in 7:41.42. "I didn\'t want to sit back and get out-kicked," said Cragg. "So I kept on the pace. The plan was to go with 500m to go no matter what, but when Bekele made the mistake that was it. The race was mine." Sweden\'s Carolina Kluft, the Olympic heptathlon champion, and Slovenia\'s Jolanda Ceplak had winning performances, too. Kluft took the long jump at 6.63m, while Ceplak easily won the women\'s 800m in 2:01.52.\n'

In [6]:
directory_path = '/content/BBC News Summary/Summaries/sport'

# Initialize lists to store data
file_names = []
file_contents = []

# Loop through files in the directory
for file_name in os.listdir(directory_path):
    if file_name.endswith('.txt'):
        file_names.append(file_name)
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'rb') as file:
            content = file.read().decode(errors='ignore')
            file_contents.append(content)

# Create a DataFrame
data = {'File Name': file_names, 'File_Summary': file_contents}
df2 = pd.DataFrame(data)

In [7]:
df2['File_Summary'][3]

"Ethiopia's Tirunesh Dibaba set a new world record in winning the women's 5,000m at the Boston Indoor Games.The plan was to go with 500m to go no matter what, but when Bekele made the mistake that was it.Dibaba won in 14 minutes 32.93 seconds to erase the previous world indoor mark of 14:39.29 set by another Ethiopian, Berhane Adera, in Stuttgart last year.Ireland's Alistair Cragg won in 7:39.89 as Bekele battled to second in 7:41.42."

In [8]:
dataset = pd.merge(df1, df2, on='File Name').drop(['File Name'], axis= 1)
dataset

,File_Content,File_Summary
0,Young debut cut short by Ginepri\n\nFifteen-ye...,"Young, who beat two players ranked in the top ..."
1,Owen set for skipper role\n\nWales number eigh...,"""Gareth is going to be a massive loss because ..."
2,England claim Dubai Sevens glory\n\nEngland be...,England beat Fiji 26-21 in a dramatic final in...
3,"Dibaba breaks 5,000m world record\n\nEthiopia'...",Ethiopia's Tirunesh Dibaba set a new world rec...
4,Hitzlsperger waiting on O'Leary\n\nAston Villa...,Aston Villa's Thomas Hitzlsperger is waiting t...
...,...,...
506,Souness backs Smith for Scotland\n\nGraeme Sou...,Graeme Souness believes Walter Smith would be ...
507,Campbell rescues Arsenal\n\nSol Campbell prove...,"Almunia, Lauren, Toure, Campbell, Cole, Pires,..."
508,Player burn-out worries Robinson\n\nEngland co...,England coach Andy Robinson says English rugby...
509,Robertson out to retain Euro lure\n\nHearts ma...,Hearts manager John Robertson hopes a place in...


In [9]:
dataset['File_Title']= "Sports"
dataset

,File_Content,File_Summary,File_Title
0,Young debut cut short by Ginepri\n\nFifteen-ye...,"Young, who beat two players ranked in the top ...",Sports
1,Owen set for skipper role\n\nWales number eigh...,"""Gareth is going to be a massive loss because ...",Sports
2,England claim Dubai Sevens glory\n\nEngland be...,England beat Fiji 26-21 in a dramatic final in...,Sports
3,"Dibaba breaks 5,000m world record\n\nEthiopia'...",Ethiopia's Tirunesh Dibaba set a new world rec...,Sports
4,Hitzlsperger waiting on O'Leary\n\nAston Villa...,Aston Villa's Thomas Hitzlsperger is waiting t...,Sports
...,...,...,...
506,Souness backs Smith for Scotland\n\nGraeme Sou...,Graeme Souness believes Walter Smith would be ...,Sports
507,Campbell rescues Arsenal\n\nSol Campbell prove...,"Almunia, Lauren, Toure, Campbell, Cole, Pires,...",Sports
508,Player burn-out worries Robinson\n\nEngland co...,England coach Andy Robinson says English rugby...,Sports
509,Robertson out to retain Euro lure\n\nHearts ma...,Hearts manager John Robertson hopes a place in...,Sports


In [10]:
dataset.isna().sum()

File_Content    0
File_Summary    0
File_Title      0
dtype: int64

In [11]:
from datasets import Dataset

final_dataset= Dataset.from_pandas(dataset)

In [12]:
nlp_data = final_dataset.train_test_split(test_size=0.2)

In [13]:
checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [14]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["File_Content"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(text_target=examples["File_Summary"], max_length=224, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
tokenized_dataset = nlp_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [17]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

### Rouge Evaluation

In [18]:
import evaluate

rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

### Training  

In [19]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_text_summarizer",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args= training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)


In [21]:
import torch
torch.cuda.empty_cache()
!nvidia-smi

Mon Aug 21 10:27:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |   1819MiB / 15360MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.548498,0.181500,0.148500,0.173100,0.173700,19.000000
2,No log,0.501017,0.182600,0.152800,0.177800,0.178200,19.000000
3,1.181600,0.478216,0.185200,0.155700,0.179800,0.180200,19.000000
4,1.181600,0.464008,0.181800,0.152200,0.175600,0.176000,19.000000
5,0.555100,0.457037,0.179700,0.149000,0.173600,0.174400,19.000000
6,0.555100,0.450184,0.187100,0.158400,0.181500,0.182200,19.000000
7,0.555100,0.447441,0.186500,0.157400,0.181600,0.182200,19.000000
8,0.503200,0.445459,0.190000,0.164000,0.186800,0.186800,19.000000
9,0.503200,0.444689,0.191400,0.165200,0.187300,0.187700,19.000000
10,0.472400,0.444537,0.191600,0.165500,0.187400,0.187800,19.000000


TrainOutput(global_step=2040, training_loss=0.6740307274986715, metrics={'train_runtime': 902.2175, 'train_samples_per_second': 4.522, 'train_steps_per_second': 2.261, 'total_flos': 2212872097075200.0, 'train_loss': 0.6740307274986715, 'epoch': 10.0})

### Teseting

In [23]:
test_text= " summarize: Why is this deal historic?\
It is just another club for Lionel Messi to go to and show his skills but it is indeed \
historic for the MLS and American Football as a whole. This will bring them the most eyeballs \
they have ever seen. The social media following of Inter Miami has risen by more than 300% and \
the ticket price for all the matches of Inter Miami have gone from $50 to $10,000 . The amount \
of worldwide attention that the MLS will get from this one signing will be humongous.\
Lionel Messi will bring fans to the Stadiums and Inter Miami will have to move \
from their 18,000 seater stadium to the Hard Rock Café Stadium which has a capacity of 70,000 because \
that is the standard of Messi. The sponsors that both Inter Miami and the league will get are \
unfathomable. It is like Michael Jordan playing in the Spanish Basketball League. All the celebrities \
will line up to watch him play and new TV and streaming deals will be made with all the countries around \
the world like India."

In [24]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/my_text_summarizer/checkpoint-2000")
inputs = tokenizer(test_text, return_tensors="pt").input_ids

In [25]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/my_text_summarizer/checkpoint-2000")
outputs = model.generate(inputs, max_new_tokens=500, do_sample=False)

In [26]:
summary= tokenizer.decode(outputs[0], skip_special_tokens=True)
summary

'The social media following of Inter Miami has risen by more than 300% and the ticket price for all the matches of Inter Miami have gone from $50 to $10,000.Lionel Messi will bring fans to the Stadiums and Inter Miami will have to move from their 18,000 seater stadium to the Hard Rock Café Stadium which has a capacity of 70,000 because that is the standard of Messi.The sponsors that both Inter Miami and the league will get are unfathomable.'

                                                                                                  ### Pranjal Tripathi